In [5]:
import numpy as np  
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from keras.models import Sequential, clone_model
from keras.layers import Dense  
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Flatten
import os
from sklearn.metrics import mean_squared_error, mean_absolute_error
import math

import datetime as dt
import matplotlib.dates as mdates

from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler # for scaling data into [0, 1]  


import data_base
import openpyxl

import wandb
from wandb.keras import WandbMetricsLogger, WandbModelCheckpoint
import random

In [6]:

tf.config.list_physical_devices('GPU')

[]

In [7]:
!wandb login d88f8e394954db0905efd5e45c2e5108f96a714a

wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\marti/.netrc


In [8]:
real_df = pd.read_csv('real_df_IA.csv')
real_df['date_time'] = pd.to_datetime(real_df['date_time'])
real_df.set_index('date_time', inplace=True)


start = '2021-06-26'
end = '2023-05-15'
weather_data = real_df.loc[start:end]
weather_data = weather_data.drop('GHI', axis=1)
weather_data = weather_data.drop('GTI', axis=1)
weather_data.loc[weather_data['PAC'] > 38000, 'PAC'] = 38000

weather_data = weather_data.resample('H').mean()
weather_data = weather_data.fillna(0)
df = weather_data

# VAMOS A CREAR LA FUNCIÓN PARA WEIGHTS

In [5]:
def entrenar_y_evaluar(config=None):
    wandb.init(config=config)
    config = wandb.config
        
    ventana_temporal = config.ventana_temporal
    caracteristicas_lstm = config.caracteristicas_lstm
    ratio_dropout = config.ratio_dropout
    tamano_batch = 2048
    learning_rate_in = 0.01
    numero_de_epoch = 250


    
#     SE SEPARAN LOS DATOS
    PorcentajeEntradas = 80.55
    # Número de filas.
    nFilas = df.shape[0] 
    # Cantidad de datos de entrenamiento
    noEntrenamiento = int((PorcentajeEntradas / 100) * nFilas) 
    # Tomamos los datos de entrenemiento
    datos_entrenamiento = df[:noEntrenamiento]
    # Tomamos los datos de test
    datos_test = df[(noEntrenamiento + 1):]
    
#     SELECCIONAMOS LOS PREDICTORES
    #PAC, GHI, TA, WS.

    PredictoresSelecionados = ['PAC','GHI_PYR','TA']
        


    # Solo tenemos un predictor 
    NoPredictoresSelecionados = len(PredictoresSelecionados)
    # Tomamos los indices
    posPV = PredictoresSelecionados.index(PredictoresSelecionados[0]) 
    # Tomamos los valores de los predictores seleccionados para el conjunto de Entrenamiento
    procesados_entrenamiento = datos_entrenamiento[PredictoresSelecionados].values 
    # Tomamos los valores de los predictores seleccionados para el conjunto de test
    procesados_test = datos_test[PredictoresSelecionados].values
    
#   AJUSTAMOS LOS VALORES A 0-1
    scaler = MinMaxScaler(feature_range = (0, 1)) 
    scaler_output = MinMaxScaler(feature_range = (0, 1)) 
    escalados_entrenamiento = scaler.fit_transform(procesados_entrenamiento) # scales the values into [0, 1]
    scaler_output.fit(procesados_entrenamiento[:,0].reshape(-1,1)) # scales the values into [0, 1]
#   SE AJUSTAN LOS DATOS DE T = N + 1
    features_set = []
    labels = []
    max_distance = 24
     
    for i in range(ventana_temporal, len(escalados_entrenamiento)):  
        features_set.append(escalados_entrenamiento[i - ventana_temporal:i])
        labels.append(escalados_entrenamiento[i, posPV])
    labels_direct = []
    for i in range(ventana_temporal, len(escalados_entrenamiento) - max_distance):  
        labels_direct.append(tf.convert_to_tensor((escalados_entrenamiento[i:i+max_distance, posPV]), dtype=tf.float32))
    labels_direct = tf.stack(labels_direct)

    # Convertimos las listas a arrays
    features_set, labels = np.array(features_set), np.array(labels) 
    features_set = np.reshape(features_set, (features_set.shape[0], features_set.shape[1], NoPredictoresSelecionados))
    
#     REPETIMOS PARA EL CONJUNTO DE TEST

    # take the dates of the test data for future plotting
    # fecha_test = procesados_test
    # fecha_test = [onlyDate for [onlyDate] in fecha_test]

    # take also the previous hoursBack from training in order to predict the first test hours    
    total = pd.concat((datos_entrenamiento[PredictoresSelecionados], datos_test[PredictoresSelecionados]), axis=0)

    entradas_test = total[len(total) - len(procesados_test) - ventana_temporal:].values          
    entradas_test = entradas_test.reshape(-1, NoPredictoresSelecionados) # puts every item between brackets, values are unchanged
    entradas_test_transform = scaler.transform(entradas_test) # also scale the test data 

    # collect the test feature set as previously for training
    test_features = []
    for i in range(ventana_temporal, len(procesados_test) + ventana_temporal):  
        test_features.append(entradas_test_transform[i-ventana_temporal:i])

    # prepare test in numpy array and format for LSTM
    test_features = np.array(test_features)
    test_features = np.reshape(test_features, (test_features.shape[0], test_features.shape[1], NoPredictoresSelecionados))  
    
    
#     NECESITAMOS LA SALIDA HECHA EN 24H

    test_features_output = []
    for i in range(24, len(procesados_test) + 24):  
        test_features_output.append(entradas_test_transform[i-24:i])

    # prepare test in numpy array and format for LSTM
    test_features_output = np.array(test_features_output)
    test_features_output = np.reshape(test_features_output, (test_features_output.shape[0], test_features_output.shape[1], NoPredictoresSelecionados))

#     CREAMOS EL MODELO SECUENCIAL EN KERAS
    model = Sequential() 
    
    # Arquitectura = 2 Modelos conectados

    model.add(LSTM(units= caracteristicas_lstm, return_sequences=True, input_shape=(features_set.shape[1], NoPredictoresSelecionados)))
    # numero de unidades = el tamaño de las caracteristicas del estado de la celda (h_t)
    # return_sequences = veradero para concatenar capas LSTM
    # Para la primera LSTM el numero de entradas se tiene que especificar
    # La forma de entrada es 1 ó más registros, el tamaño de la ventana temporal y el número de predictores.

    # Se desactivan aleatoriamente un % de los nodos LSTM
    model.add(Dropout(ratio_dropout)) 

    # Segunda capa LSTM
    model.add(LSTM(units=caracteristicas_lstm))  
    model.add(Dropout(ratio_dropout)) 

    # Añadimos una capa totalmente conectada
    # units = 1 Solo queremos predecir la siguiente hora.
    model.add(Dense(units = max_distance, activation = 'sigmoid')) 
    
#     SE SELECCIONA OPTIMIZER Y FUNCIÓND DE PÉRDIDAS


    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate_in)
    model.compile(optimizer = optimizer, loss = 'mean_absolute_error')

#    SE ENTRENA EL MODELO Y SE SACAN DATOS RELEVANTES
    callback_es = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10)
    history = model.fit(features_set[0:len(labels_direct)], labels_direct,  epochs = numero_de_epoch,callbacks = [callback_es, WandbMetricsLogger()], batch_size = tamano_batch)
    predictions = model.predict(test_features)
    labels_test = test_features_output[max_distance:,:,0].squeeze()
    predictions = predictions[0:len(labels_test)]
    predictions_0 = predictions[:,0].reshape(-1,1)
    labels_test_0 = labels_test[:,0].reshape(-1,1)
    predictions_23 = predictions[:,23].reshape(-1,1)
    labels_test_23 = labels_test[:,23].reshape(-1,1)
    
    RMSE = math.sqrt(mean_squared_error(scaler_output.inverse_transform(predictions),scaler_output.inverse_transform(labels_test) ))
    MAE = mean_absolute_error(scaler_output.inverse_transform(predictions),scaler_output.inverse_transform(labels_test) )
    RMSE_0 =  math.sqrt(mean_squared_error(scaler_output.inverse_transform(predictions_0),scaler_output.inverse_transform(labels_test_0) ))
    MAE_0 = mean_absolute_error(scaler_output.inverse_transform(predictions_0),scaler_output.inverse_transform(labels_test_0) )
    RMSE_23 =  math.sqrt(mean_squared_error(scaler_output.inverse_transform(predictions_23),scaler_output.inverse_transform(labels_test_23) ))
    MAE_23 = mean_absolute_error(scaler_output.inverse_transform(predictions_23),scaler_output.inverse_transform(labels_test_23) )
    
    loss = history.history['loss'][-1]  # Obtener las métricas de pérdida del historial
    # wandb.log({"mae": mae, "rmse": rmse, "rmse_relative": rmse_relative})
    wandb.log({"MAE": MAE, "RMSE": RMSE, "MAE_0": MAE_0,"RMSE_0": RMSE_0,"MAE_23": MAE_23, "RMSE_23": RMSE_23, "loss": loss})   

In [6]:
sweep_config = {
    'method': 'grid'
    }
parameters_dict = {
    'count': {
        'values': [0,1,2]
        },
    'ventana_temporal': {
        'values': [72]
        },
    'caracteristicas_lstm': {
          'values': [24]
        },   
    'ratio_dropout': {
          'values': [0.35]
        },   
    }

sweep_config['parameters'] = parameters_dict

In [7]:
sweep_id = wandb.sweep(sweep_config, project="Test_carlos_1")

Create sweep with ID: s5u3fy86
Sweep URL: https://wandb.ai/martinkek/Test_carlos_1/sweeps/s5u3fy86


In [ ]:
wandb.agent(sweep_id, entrenar_y_evaluar)

wandb: Agent Starting Run: uekbxn1x with config:
wandb: 	caracteristicas_lstm: 24
wandb: 	count: 0
wandb: 	ratio_dropout: 0.35
wandb: 	ventana_temporal: 72
wandb: Currently logged in as: martinmartinruiz (martinkek). Use `wandb login --relogin` to force relogin


Epoch 1/250
7/7 [==============================] - 4s 238ms/step - loss: 0.3580
Epoch 2/250
7/7 [==============================] - 2s 229ms/step - loss: 0.2846
Epoch 3/250
7/7 [==============================] - 2s 229ms/step - loss: 0.2677
Epoch 4/250
7/7 [==============================] - 2s 229ms/step - loss: 0.2656
Epoch 5/250
7/7 [==============================] - 2s 237ms/step - loss: 0.2654
Epoch 6/250
7/7 [==============================] - 2s 234ms/step - loss: 0.2653
Epoch 7/250
7/7 [==============================] - 2s 231ms/step - loss: 0.2653
Epoch 8/250
7/7 [==============================] - 2s 238ms/step - loss: 0.2653
Epoch 9/250
7/7 [==============================] - 2s 236ms/step - loss: 0.2653
Epoch 10/250
4/7 [================>.............] - ETA: 0s - loss: 0.2654